In [1]:
#%pip install tensorflow

In [11]:
import pandas as pd
import numpy as np
import re
import tensorflow
from tensorflow import keras
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed, Dropout,CuDNNLSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [10]:
# Read data that I cleared in Preprocess.ipynb and organize it in pairs of "questions" and "answers"

data = pd.read_csv('punc_preprocessed.csv')
pairs = list(zip(data.text,data.response))
print(pairs[0])
print("Length of pairs: {}".format(len(pairs)))

('ох я рад тебя видеть здесь', 'прекрасно значит я здесь буду')
Length of pairs: 7593


In [19]:
# Making lists of tokens for questions and answers. For answers I will also add <START> and <END> 
#   tokens at both ends of each sentence. 

input_docs = []
target_docs = []
all_tokens = set()

for line in pairs[:4000]:
    input_doc, target_doc = str(line[0]), str(line[1])
    input_docs.append(input_doc)
    target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
    target_doc = '<START> '+target_doc+' <END>'
    target_docs.append(target_doc)

    for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
        if token not in all_tokens:
            all_tokens.add(token)
    for token in target_doc.split():
        if token not in all_tokens:
            all_tokens.add(token)

all_tokens.add('<UNK>')

In [20]:
# Creating 2 dicts {token:idx} and {idx:token} for convert sentences to tokenized and then padded sequences

word2index = dict([(token, i) for i, token in enumerate(all_tokens)])
index2word = dict([(i,token) for i, token in enumerate(all_tokens)])
VOCAB_SIZE = len(word2index) + 1
print('Vocabulary size : {}'.format(VOCAB_SIZE))

maxlen_questions = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
maxlen_answers = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])
MAX_LEN = 13

tokenized_questions = []
tokenized_answers = []
for i in input_docs:
    tokenized_questions.append([word2index[token] if token in word2index else word2index['<UNK>'] for token in i.split(' ')])
for i in target_docs:
    tokenized_answers.append([word2index[token] if token in word2index else word2index['<UNK>'] for token in i.split(' ')])

padded_questions = pad_sequences(tokenized_questions, maxlen=MAX_LEN,padding='post', truncating='post')
padded_answers = pad_sequences(tokenized_answers, maxlen=MAX_LEN,padding='post', truncating='post')

Vocabulary size : 8620


In [21]:
encoder_input_data = padded_questions
decoder_input_data = padded_answers

decoder_target_data = []
for i in decoder_input_data:
    decoder_target_data.append(i[1:])

decoder_target_data = pad_sequences(decoder_target_data, maxlen=MAX_LEN,padding='post', truncating='post')
decoder_target_data = keras.utils.to_categorical(decoder_target_data, VOCAB_SIZE)

In [22]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(4000, 13)
(4000, 13)
(4000, 13, 8620)


In [23]:
#---------------------------------------------------------------
dimensionality = 256
batch_size = 32
epochs = 300

encoder_inputs = Input(shape=(MAX_LEN,))
decoder_inputs = Input(shape=(MAX_LEN,))

embed = Embedding(VOCAB_SIZE+1, output_dim=50, input_length=MAX_LEN, trainable=True)

encoder_embedding = embed(encoder_inputs)
encoder_LSTM = LSTM(dimensionality, return_sequences=True,return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

decoder_embedding = embed(decoder_inputs)
decoder_LSTM = LSTM(dimensionality,return_sequences=True,return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#keras.utils.plot_model(model, show_shapes=True)

In [24]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

Epoch 1/300
100/100 [==============================] - 14s 118ms/step - loss: 4.0896 - accuracy: 0.5408 - val_loss: 3.4767 - val_accuracy: 0.5405
Epoch 2/300
100/100 [==============================] - 10s 104ms/step - loss: 3.0590 - accuracy: 0.5600 - val_loss: 3.2818 - val_accuracy: 0.5596
Epoch 3/300
100/100 [==============================] - 10s 104ms/step - loss: 2.6871 - accuracy: 0.6083 - val_loss: 3.0691 - val_accuracy: 0.6009
Epoch 4/300
100/100 [==============================] - 10s 104ms/step - loss: 2.5995 - accuracy: 0.6179 - val_loss: 3.0670 - val_accuracy: 0.6024
Epoch 5/300
100/100 [==============================] - 10s 104ms/step - loss: 2.5490 - accuracy: 0.6189 - val_loss: 3.0935 - val_accuracy: 0.6024
Epoch 6/300
100/100 [==============================] - 11s 106ms/step - loss: 2.5083 - accuracy: 0.6193 - val_loss: 3.1031 - val_accuracy: 0.6035
Epoch 7/300
100/100 [==============================] - 12s 117ms/step - loss: 2.4758 - accuracy: 0.6194 - val_loss: 3.1227 -

In [31]:
encoder_model = Model([encoder_inputs], [state_h, state_c])

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]

decoder_outputs, state_hidden, state_cell = decoder_LSTM(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [46]:

def str_to_tokens(sentence: str):
    words = sentence.lower().split()
    tokens_list = list()
    for current_word in words:
        result = word2index[current_word]
        if result != '':
            tokens_list.append(result)
    return pad_sequences([tokens_list], maxlen=MAX_LEN, padding='post')


while True:
    user_input = input()
    if user_input == 'пока':
        break
    prepro = [user_input]
    txt = []
    for x in prepro:
        tokens_list = []
        for i in x.split():
            try: tokens_list.append(word2index[i])
            except: tokens_list.append(word2index['<UNK>'])
        txt.append(tokens_list)
    txt = pad_sequences(txt, maxlen=MAX_LEN, padding='post')
    states_values = encoder_model.predict(txt)
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = word2index['<START>']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs, h, c = decoder_model.predict([empty_target_seq] + states_values) 
        decoder_concat_input = decoder_dense(dec_outputs)        
        sampled_word_index = np.argmax(decoder_concat_input[0, -1, :])
        sampled_word = index2word[sampled_word_index]
        if sampled_word != '<START>':
            decoded_translation += sampled_word + ' '
        if sampled_word == '<END>' or len(decoded_translation.split())>MAX_LEN:
            stop_condition = True
        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_word_index
        states_values = [h, c]
    print(decoded_translation)

 что скажешь


1/1 [==============================] - 0s 16ms/step
как работа сегодня <END> 


 хорошо а у тебя


1/1 [==============================] - 0s 15ms/step
где ты сейчас <END> 


 дома а ты


1/1 [==============================] - 0s 15ms/step
выходи там где фитнес пойдём друг другу навстречу <END> 


 пока


---
### Пример диалога:
```
- что скажешь
- как работа сегодня
- хорошо а у тебя
- дома а ты
- выходи там где фитнес пойдём друг другу навстречу

```
---